In [1]:
from __future__ import print_function

%load_ext autoreload
%autoreload 2
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Box, Layout

In [2]:
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='pearl')

In [3]:
# Import yfinance
import yfinance as yf


In [4]:
query = widgets.Text(
                        value='',
                        placeholder='Ticker here...',
                        description='Analysis of',
                        disabled=False,
                        continuous_update=False,
    )

In [5]:
stocks_init = pd.DataFrame(columns=['Date',
                                    'ticker',
                               'Open',
                               'High',
                               'Low',
                               'Close',
                               'Volume',
                               'Dividends',
                               'Stock',
                               'Splits'])

# Get the data for the stock  by specifying the stock ticker, start date, and end date
def grabstock(ticker,stocks):
        # Add suffix for the Paris market
        #ticker = ticker + '.PA'
        yfdata = yf.Ticker(ticker)
        yfdata = yfdata.history(period="max")
        if len(yfdata) == 0:
                print('>> lack '+ticker)
                return stocks
        else:
                yfdata['ticker'] = ticker[0:-3]
                # ignore_index=False permet de preserver les dates
                stocks = stocks.append(yfdata, ignore_index=False)
                return stocks
            
def response(change):
            Ticker = str(query.value)
            stocks = grabstock(str(query.value),stocks_init)
            live_data = pd.DataFrame(stocks)
            ## On calcule la vitesse et l'acceleration
            ## !! pour continuer, executer d'abord ##Extract Structural Datas
            live_augmented_datas =  np.array([live_data.Close.values[0:-2],
                                  live_data.Close[1:-1].values-live_data.Close.values[0:-2],
                                  live_data.Close[2::].values-2*live_data.Close.values[1:-1]+live_data.Close[0:-2].values]
                                        )
            live_augmented_datas = np.transpose(live_augmented_datas)
            live_augmented_datas = pd.DataFrame(live_augmented_datas, columns=['Value','Velocity','Acceleration'])
            # Structure calculation
            sol = np.array([]).reshape(0,2)
            for i in range(0,len(live_augmented_datas)-1):
                    B = [live_augmented_datas.loc[i].Acceleration,
                         live_augmented_datas.loc[i+1].Acceleration]
                    A = [[live_augmented_datas.loc[i].Velocity,live_augmented_datas.loc[i].Value],
                         [live_augmented_datas.loc[i+1].Velocity,live_augmented_datas.loc[i+1].Value]]
                    if np.linalg.det(A) == 0:
                        results = [0,0]
                        sol = np.vstack((sol,results))
                    else:
                        results = np.linalg.solve(A,B)
                        sol = np.vstack((sol,results))
            structural_datas = pd.DataFrame(sol, columns=['Friction','Stiffness'])
            structural_datas['Delta'] = np.sqrt(structural_datas['Friction']*structural_datas['Friction']/4-structural_datas['Stiffness'])
            live_augmented_datas = live_augmented_datas.join(structural_datas)
            live_augmented_datas = live_augmented_datas.fillna(0)
            live_augmented_datas['Index'] = live_data.index[0:-2]
            with fig1.batch_update():
                fig3.data[0].x = live_augmented_datas.Index
                fig3.data[0].y = live_augmented_datas.Value
                fig2.data[0].x = live_augmented_datas.Value
                fig2.data[0].y = live_augmented_datas.Velocity
                fig2.data[0].marker.color = live_augmented_datas.index
                fig1.data[0].header.values = list(live_augmented_datas.columns)
                fig1.data[0].cells.values = [live_augmented_datas[k].tolist() for k in live_augmented_datas.columns]
        
           # [live_augmented_datas.Velocity.eq(0)]

In [6]:

trace1 = go.Scatter(x = stocks_init.Date,  
               y = stocks_init.Close,
              # marker=dict(
              #          color = live_augmented_datas['Friction'],
              #         size = abs(live_augmented_datas['Velocity']),
              #          colorscale='Viridis',
              #      ),
               mode = 'lines',
    )
fig3 = go.FigureWidget(data=[trace1],
                    )
fig3.update_layout(title_text = " Stock Price history",
      xaxis=dict(

        rangeslider=dict(
            visible=True
            ),
        type="date"
    ),
         )
query.observe(response, names='value')
VBox([query,fig3])

c:\users\alan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\series.py:679: RuntimeWarning:

invalid value encountered in sqrt



In [7]:
trace2 = go.Scatter( x=stocks_init.Close, 
                     y=stocks_init.High,
                     #color= stocks_init.index,
                     #size=abs(live_augmented_datas.Stiffness),
                     #color_continuous_scale = 'plasma',
                     #marginal_y="violin",
                     #marginal_x="violin",
                     mode = 'markers',
                  )
fig2 = go.FigureWidget(data=[trace2],
                    )

fig2.update_layout(title_text = " Value and Trend over time",
                  #xaxis_rangeslider_visible=True,
                 )

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scatter',
              'uid': '1f9fae7…

In [8]:
trace3 = go.Table(
    header=dict(values=list(['Value', 'Friction', 'Stiffness']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[stocks_init.Close, stocks_init.Volume, stocks_init.Date],
               fill_color='lavender',
               align='left'))

fig1 = go.FigureWidget(data=[trace3],
                    )
fig1.update_layout(title_text = " Buy&Sell Values",
                  #xaxis_rangeslider_visible=True,
                 )

FigureWidget({
    'data': [{'cells': {'align': 'left', 'fill': {'color': 'lavender'}, 'values': [[], [], []]}…